In [1]:
import sys
import os
import IPython as IP
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pickle
import helpers
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from IPython.display import clear_output, display
from scipy.stats import kurtosis, skew
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading Meta_Data

In [3]:
# Set your path to the dataset
us8k_path = os.path.abspath('C:/Users/VSBAG/Desktop/DSE_Milan/3rd_sem_subject/Machine Learning/Project/Sound_classification/UrbanSound8K')
audio_path = os.path.join(us8k_path, 'audio')
metadata_path = os.path.join(us8k_path, 'metadata/UrbanSound8K.csv')

In [4]:
# Load the metadata from the generated CSV
metadata = pd.read_csv(metadata_path)
print(metadata.shape)
metadata.head(10)

(8732, 8)


,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.000000,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.500000,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.500000,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.000000,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.500000,72.500000,1,5,2,children_playing
5,100263-2-0-143.wav,100263,71.500000,75.500000,1,5,2,children_playing
6,100263-2-0-161.wav,100263,80.500000,84.500000,1,5,2,children_playing
7,100263-2-0-3.wav,100263,1.500000,5.500000,1,5,2,children_playing
8,100263-2-0-36.wav,100263,18.000000,22.000000,1,5,2,children_playing
9,100648-1-0-0.wav,100648,4.823402,5.471927,2,10,1,car_horn


In [7]:
for index, row in metadata.iterrows():
    file_path = os.path.join(os.path.abspath(audio_path), 'fold' + str(row["fold"]), str(row["slice_file_name"]))
    class_label = row["class"]

In [13]:
#feature set
y,sr=librosa.load(file_path)
melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
mel1 = np.max(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
mel2 = np.min(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
mel3 = np.var(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
mel4 = np.median(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
mfccs_min = np.min(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
mfccs_min.resize((40,),refcheck=False)
mfccs_max = np.max(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
mfccs_max.resize((40,),refcheck=False)
mfccs_median = np.median(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 
mfccs_median.resize((40,),refcheck=False)
mfccs_mean = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
mfccs_mean.resize((40,),refcheck=False)
mfccs_variance = np.var(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 
mfccs_variance.resize((40,),refcheck=False)
mfccs_skewness = skew(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
mfccs_skewness.resize((40,),refcheck=False)
mfccs_kurtosis = kurtosis(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 
mfccs_kurtosis.resize((40,),refcheck=False)
chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
chrom2=np.std(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
rmse1 = np.mean(librosa.feature.rms(y=y).T,axis=0)
rmse1.resize((40,),refcheck=False)
rmse2 = np.std(librosa.feature.rms(y=y).T,axis=0)
rmse2.resize((40,),refcheck=False)
spec_bw1 = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
spec_bw1.resize((40,),refcheck=False)
spec_bw2 = np.std(librosa.feature.spectral_bandwidth(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
spec_bw2.resize((40,),refcheck=False)
rolloff1 = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
rolloff1.resize((40,),refcheck=False)
rolloff2 = np.std(librosa.feature.spectral_rolloff(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
rolloff2.resize((40,),refcheck=False)
centroid1 = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
centroid1.resize((40,),refcheck=False)
centroid2 = np.std(librosa.feature.spectral_centroid(y=y, sr=sr,n_fft=2048,hop_length=512).T,axis=0)
centroid2.resize((40,),refcheck=False)
zcr1 = np.mean(librosa.feature.zero_crossing_rate(y).T,axis=0)
zcr1.resize((40,),refcheck=False)
zcr2 = np.std(librosa.feature.zero_crossing_rate(y).T,axis=0)
zcr2.resize((40,),refcheck=False)
chrcn1=np.mean(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
chrcn1.resize((40,),refcheck=False)
chrcn2=np.std(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
chrcn2.resize((40,),refcheck=False)
chrcqt1=np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
chrcqt1.resize((40,),refcheck=False)
chrcqt2=np.std(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
chrcqt2.resize((40,),refcheck=False)
melspectrogram.shape,mel1.shape,mel2.shape,mel3.shape,mel4.shape,mfccs_min.shape,mfccs_max.shape,mfccs_median.shape,mfccs_mean.shape,mfccs_variance.shape,mfccs_skewness.shape,mfccs_kurtosis.shape,chroma_stft.shape,chrom2.shape,rmse1.shape,rmse2.shape,spec_bw1.shape,spec_bw2.shape,rolloff1.shape,rolloff2.shape,centroid1.shape,centroid2.shape,zcr1.shape,zcr2.shape,chrcn1.shape,chrcn2.shape,chrcqt1.shape,chrcqt2.shape

((40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,),
 (40,))

In [14]:
features=np.vstack((melspectrogram,mel1,mel2,mel3,mel4,mfccs_min,mfccs_max,mfccs_median,mfccs_mean,mfccs_variance,mfccs_skewness,mfccs_kurtosis,chroma_stft,chrom2,rmse1,rmse2,spec_bw1,spec_bw2,rolloff1,rolloff2,centroid1,centroid2,zcr1,zcr2,chrcn1,chrcn2,chrcqt1,chrcqt2))
features.reshape

<function ndarray.reshape>

In [15]:
features

array([[8.71442780e-02, 1.68030888e-01, 1.96782798e-01, ...,
        1.63907779e-03, 7.97315151e-04, 3.04472516e-04],
       [1.91622913e-01, 4.80677158e-01, 5.65517724e-01, ...,
        8.93815234e-03, 3.18803638e-03, 1.00509473e-03],
       [2.44497098e-02, 4.04719859e-02, 5.08622006e-02, ...,
        1.41392768e-04, 6.73082686e-05, 3.84440827e-05],
       ...,
       [4.85125070e-02, 7.21354530e-02, 6.41692204e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.32418096e-01, 5.76046646e-01, 6.38002634e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.85125070e-02, 7.21354530e-02, 6.41692204e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [17]:
#preprocessing using entire feature set
from tqdm import tqdm
x_train=[]
x_test=[]
y_train=[]
y_test=[]
path="C:/Users/VSBAG/Desktop/DSE_Milan/3rd_sem_subject/Machine Learning/Project/Sound_classification/UrbanSound8K/audio/fold"
for i in tqdm(range(len(meta_data))):
    fold_no=str(meta_data.iloc[i]["fold"])
    file=meta_data.iloc[i]["slice_file_name"]
    label=meta_data.iloc[i]["classID"]
    filename=path+fold_no+"/"+file
    y,sr=librosa.load(filename)
    mfccs_min = np.min(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
    mfccs_max = np.max(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  
    mfccs_median = np.median(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 

    mfccs_mean = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  

    mfccs_variance = np.var(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 

    mfccs_skewness = skew(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0)  

    mfccs_kurtosis = kurtosis(librosa.feature.mfcc(y=y, sr=sr, n_mfcc = 20).T,axis=0) 
    melspectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    mel1 = np.max(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    mel2 = np.min(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    mel3 = np.var(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    mel4 = np.median(librosa.feature.melspectrogram(y=y, sr=sr, n_mels=40,fmax=8000).T,axis=0)
    chroma_stft=np.mean(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    chrom2=np.std(librosa.feature.chroma_stft(y=y, sr=sr,n_chroma=40).T,axis=0)
    rmse1 = np.mean(librosa.feature.rms(y=y).T,axis=0)
    rmse2 = np.std(librosa.feature.rms(y=y).T,axis=0)
    spec_bw1 = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr).T,axis=0)
    spec_bw2 = np.std(librosa.feature.spectral_bandwidth(y=y, sr=sr).T,axis=0)
    rolloff1 = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr).T,axis=0)
    rolloff2 = np.std(librosa.feature.spectral_rolloff(y=y, sr=sr).T,axis=0)
    centroid1 = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr).T,axis=0)
    centroid2 = np.std(librosa.feature.spectral_centroid(y=y, sr=sr).T,axis=0)
    zcr1 = np.mean(librosa.feature.zero_crossing_rate(y).T,axis=0)
    zcr2 = np.std(librosa.feature.zero_crossing_rate(y).T,axis=0)
    chrcn1=np.mean(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
    chrcn2=np.std(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
    chrcqt1=np.mean(librosa.feature.chroma_cqt(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
    chrcqt2=np.std(librosa.feature.chroma_cens(y=y, sr=sr,C=None,n_chroma=12,bins_per_octave=36).T,axis=0)
    features
    if(fold_no!='10'):
      x_train.append(features)
      y_train.append(label)
    else:
      x_test.append(features)
      y_test.append(label)

100%|███████████████████████████████████████████████████████████████████████████| 8732/8732 [12:47:59<00:00,  5.28s/it]


In [18]:
len(x_train)+len(x_test)

8732

In [19]:
len(meta_data)

8732

In [20]:
#converting the lists into numpy arrays
x_train1=np.array(x_train)
x_test1=np.array(x_test)
y_train1=np.array(y_train)
y_test1=np.array(y_test)
x_train1.shape,x_test1.shape,y_train1.shape,y_test1.shape

((7895, 28, 40), (837, 28, 40), (7895,), (837,))

In [21]:
#reshaping into 2d to save in csv format
x_train_2d=np.reshape(x_train1,(x_train1.shape[0],x_train1.shape[1]*x_train1.shape[2]))
x_test_2d=np.reshape(x_test1,(x_test1.shape[0],x_test1.shape[1]*x_test1.shape[2]))
x_train_2d.shape,x_test_2d.shape

((7895, 1120), (837, 1120))

In [22]:
#saving the data numpy arrays
np.savetxt("train_data1.csv", x_train_2d, delimiter=",")
np.savetxt("test_data1.csv",x_test_2d,delimiter=",")
np.savetxt("train_labels1.csv",y_train,delimiter=",")
np.savetxt("test_labels1.csv",y_test,delimiter=",")